# 2. Exploration

In [2]:
try(library(tidyverse), silent=TRUE)
library(lubridate)
library(glue)
library(cowplot)
library(survminer)
library(survival)
library(ggsci)
library(arsenal)
library(yaml)

#setwd("/")
#path = "/home/steinfej/projects/uk_biobank/"
#dataset_path = "data/datasets/cvd_massive_excl_emb_ind"

Warning message:
“package ‘tidyverse’ was built under R version 4.0.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.0.5”
Warning message:
“package ‘readr’ was built under R version 4.0.5”
Warning message:
“package ‘purrr’ was built under R version 4.0.3”
Warning message:
“package ‘dplyr’ was built under R version 4.0.5”
Warning message:
“package ‘stringr’ was built under R version 4.0.5”
Warning message:
“package ‘forcats’ was built under R version 4.0.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘lubridate’ was built under R version 4.0.3”

Attaching package: ‘lubridate’


The following objects are mask

In [3]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

project_label="22_medical_records"
project_path = glue("{base_path}/results/projects/{project_label}")
figure_path = glue("{project_path}/figures")
output_path = glue("{project_path}/data")

experiment = 220531
experiment_path = glue("{output_path}/{experiment}")

[1] "/sc-projects/sc-proj-ukb-cvd"


## Load data

In [4]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [5]:
data_outcomes = arrow::read_feather(glue("{output_path}/baseline_outcomes_220531.feather", as_data_frame=FALSE))

In [6]:
dataset_path = "/sc-projects/sc-proj-ukb-cvd/data/3_datasets_post/210616_centres_dask"

In [7]:
data = arrow::read_feather(glue("{dataset_path}/data_merged.feather")) %>% filter(eid %in% unique(data_outcomes$eid)) %>% select(!ends_with("_event")&!ends_with("_event_time"))
data_description = arrow::read_feather(glue("{dataset_path}/description.feather"))

In [8]:
data = data %>% 
    mutate(erectile_dysfunction = case_when(sex=="Female" ~ FALSE, TRUE ~ erectile_dysfunction))

In [9]:
data = data %>% mutate_at(c("sex", "overall_health_rating", "smoking_status", "ethnic_background"), as.factor)
data = data %>% mutate(sex=fct_relevel(sex, c("Male", "Female")),
                       overall_health_rating=fct_relevel(overall_health_rating, c("Excellent", "Good", "Fair", "Poor")),
                       smoking_status=fct_relevel(smoking_status, c("Current", "Previous", "Never")))

In [10]:
f = list()
f$basics = c('age_at_recruitment','sex', 'ethnic_background',"townsend_deprivation_index_at_recruitment")
f$questionnaire = c('overall_health_rating','smoking_status')
f$measurements = c('body_mass_index_bmi','weight',"standing_height",'systolic_blood_pressure','diastolic_blood_pressure')
f$labs = c("cholesterol", "hdl_cholesterol", "ldl_direct","triglycerides")
f$family_history = c('fh_heart_disease')
f$diagnoses = c("diabetes1", "diabetes2", "chronic_kidney_disease", "atrial_fibrillation", "migraine", 
                      "rheumatoid_arthritis", "systemic_lupus_erythematosus", "severe_mental_illness", "erectile_dysfunction")
f$medications = c("antihypertensives", "ass", "atypical_antipsychotics", "glucocorticoids")

In [11]:
PANEL = c( # 38 clinical predictors
    # basics
    "age_at_recruitment", 
    "sex", 
    "ethnic_background",# # added
   # "education_years",
    'smoking_status', # current smoker
    #'alcohol_intake_frequency', # 'Daily or almost daily'
  #  "daily_physical_activity",

    #"daily_healthy_food",
    
    # family history
   # "fh_diabetes",
    
    # diagnoses
   # "diabetes2",
        
    # physical
    "weight", 
    "standing_height", 
    "body_mass_index_bmi",
   # 'waist_hip_ratio',
    #"waist_circumference",
    "systolic_blood_pressure",
    
    # lipids
   # "cholesterol", 
    "ldl_direct", 
    #"hdl_cholesterol",
    #"triglycerides",
    
    # diabetes
    'glucose'
   # 'glycated_haemoglobin_hba1c',
    
    # kidney
  #  'creatinine',
  #  'cystatin_c',
  #  'urea',
  #  'urate',
        
    # liver
  #  'aspartate_aminotransferase',
  #  'alanine_aminotransferase',
   # 'alkaline_phosphatase',
   # 'albumin',
    
    # inflammation
   # 'creactive_protein',
    
    # Blood counts
   # 'red_blood_cell_erythrocyte_count',
   # 'white_blood_cell_leukocyte_count',
   # 'platelet_count',
   # 'haemoglobin_concentration',
   # 'haematocrit_percentage',
   #     'mean_corpuscular_volume',
   # 'mean_corpuscular_haemoglobin',
  #  'mean_corpuscular_haemoglobin_concentration',
    
    # medications
  #  'antihypertensives'
        )

# Baseline Characteristics - Table 1

In [12]:
table_data = data %>% select(all_of(PANEL))

In [13]:
table_data

age_at_recruitment,sex,ethnic_background,smoking_status,weight,standing_height,body_mass_index_bmi,systolic_blood_pressure,ldl_direct,glucose
<dbl>,<fct>,<fct>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
49,Female,White,Current,63.8,155.0,26.5557,159.5,3.352,5.101
59,Male,White,Current,70.7,176.3,22.7465,133.0,5.389,4.835
59,Female,White,Previous,78.9,156.0,32.4211,118.5,NA,NA
63,Male,White,Previous,95.8,180.0,29.5679,141.5,3.128,5.060
51,Female,White,Never,92.3,150.0,41.0222,117.0,NA,NA
67,Male,White,Never,76.0,178.0,23.9869,138.0,2.579,5.652
60,Female,White,Never,60.1,155.0,25.0156,146.0,4.732,4.973
43,Male,White,Never,88.9,174.0,29.3632,155.0,4.076,6.158
50,Female,Black,Current,70.5,161.0,27.1980,130.5,3.329,4.688


In [14]:
library(gtsummary)
table1 = table_data %>% 
    mutate(smoking_status=case_when(smoking_status=="Current" ~ 1, TRUE ~ 0)#,
           #alcohol_intake_frequency=case_when(alcohol_intake_frequency=="Daily or almost daily" ~ 1, TRUE ~ 0)
          ) %>%
    tbl_summary(by="sex", 
                label=list(
                    age_at_recruitment ~"Age at Recruitment",
                    ethnic_background ~"Ethnicity",
                    smoking_status ~ "Current Smoker",
                    #alcohol_intake_frequency ~ "Daily Alcohol Intake",
                    body_mass_index_bmi ~ "BMI",
                    weight ~ "Weight (kg)",
                    standing_height ~ "Standing Height (cm)",
                    systolic_blood_pressure ~ "Systolic Blood Pressure (mmHg)",
                    ldl_direct ~ "LDL Cholesterol ()",
                    glucose ~ "Glucose ()"
                ),
                missing="no") %>% 
    add_overall(last=TRUE) %>% 
    bold_labels()

Warning message:
“package ‘gtsummary’ was built under R version 4.0.5”


In [15]:
library(gt)
plot_name = "Table1"
table1 %>% as_gt() %>% 
    #opt_align_table_header(align = "left") %>% 
    gt::gtsave(glue("outputs/{plot_name}.html"))
#table1 %>% as_gt()  %>% tab_header(title = "Table 1") %>% 
#    #opt_align_table_header(align = "left") %>%
 #   gt::gtsave(glue("{figure_path}/{plot_name}.html"))

Warning message:
“package ‘gt’ was built under R version 4.0.5”


# Cumulative Person Years

In [16]:
py_data =Nature Biomedical Engineering data_outcomes %>% filter(endpoint == "OMOP_4306655") %>% group_by(eid) %>% arrange(desc(time)) %>% slice(1) %>% ungroup() %>% summarise(sum_years=sum(time))

ERROR: Error in parse(text = x, srcfile = src): <text>:1:17: unexpected symbol
1: py_data =Nature Biomedical
                    ^


In [17]:
py_data 

ERROR: Error in eval(expr, envir, enclos): object 'py_data' not found


# Endpoint Characteristics

In [18]:
library(data.table)
endpoints_md = fread(glue("{experiment_path}/endpoints.csv"), colClasses = c("phecode"="character"))
endpoints = sort(endpoints_md$endpoint)


Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [19]:
endpoint_selection = c(
    "phecode_008",
    "phecode_092-2",
    "phecode_105",
    "phecode_107-2",
    "phecode_164",
    "phecode_202-2",
    "phecode_284",
    "phecode_292",
    "phecode_324-11",
    "phecode_328",
    "phecode_371",
    "phecode_401",
    "phecode_404",
    "phecode_424",
    "phecode_440-11",
    "phecode_468",
    "phecode_474",
    "phecode_522-1",
    "phecode_542-1",
    "phecode_581-1",
    "phecode_583",
    "phecode_665",
    "phecode_705-1",
    "OMOP_4306655"  
    )

In [20]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [21]:
endpoint_defs = endpoint_defs %>% 
    mutate(name = phecode_string) %>% 
    mutate(name = 
           case_when( 
               endpoint == "phecode_008"~"H. pylori", 
               endpoint == "phecode_092-2"~"Sepsis", 
               endpoint == "phecode_105"~"Breast cancer", 
                endpoint == "phecode_107-2"~"Prostate cancer", 
               endpoint == "phecode_123"~"Malignant plasma cell neoplasms", 
               endpoint == "phecode_164"~"Anemia", 
               endpoint == "phecode_200-1"~"Hypothyroidism", 
               endpoint == "phecode_232"~"Vitamin deficiencies", 
               endpoint == "phecode_284"~"Suicide attempt or self harm", 
               #endpoint == "phecode_287-5"~"Drug-induced psychosis", 
               endpoint == "phecode_324-11"~"Parkinson's",
               endpoint == "phecode_328"~"Dementia", 
               #endpoint == "phecode_404"~"Coronary heart disease", 
               endpoint == "phecode_424"~"Heart failure", 
               endpoint == "phecode_440-11"~"Deep vein thrombosis", 
               endpoint == "phecode_468"~"Pneumonia", 
               endpoint == "phecode_474"~"COPD", 
               endpoint == "phecode_518"~"Appendicitis", 
               endpoint == "phecode_542-1"~"Fibrosis and cirrhosis of liver", 
               endpoint == "phecode_583"~"Chronic kidney disease", 
               endpoint == "phecode_705-1"~"Rheumatoid arthritis", 
               endpoint == "phecode_908-1"~"(Pre)eclampsia", 
               #endpoint == "phecode_976"~"Complication of anesthesia",
               TRUE ~ name)
           )

endpoint_map = endpoint_defs$name
names(endpoint_map) =  endpoint_defs$endpoint
endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint

In [22]:
icd_to_phecode_map = fread("/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/ICD10_to_phecode_V2.csv", 
                           colClasses = c("phecode"="character")) %>%
     group_by(phecode) %>% summarise(ICD = str_c(icd10, collapse=", "))

In [23]:
endpoint_summary = endpoints_md %>% 
    #filter(endpoint %in% endpoint_selection) %>% 
    select(phecode, phecode_string, eligable, n, freq) %>%
    left_join(icd_to_phecode_map)

Joining, by = "phecode"



In [24]:
endpoint_summary %>% write_csv("outputs/endpoints.csv")